In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import plotly.express as px
import numpy as np

from pathlib import Path

import os

import glob

import pickle

In [2]:
#read in the path to the folder of interest
folder_in = 'input_data'
folder_out = 'results'

myPath = Path('..','..',f'{folder_out}', 'results_csv')
tifCounter = len(glob.glob1(myPath,"*.ilp"))
print('Infeasible scenarios:', tifCounter)

#data processing from GLUCOSE scenarios runs
#creating a database with yearly data inputs
#configfile: "config/config.yaml"

myPath = Path(f'{folder_out}','results_csv')
n_replications = int(len(glob.glob1(myPath,"*.sol")))
print("Number of replications/runs:", n_replications)
replications = list(range(0,n_replications))
print(replications)

Infeasible scenarios: 0
Number of replications/runs: 0
[]


In [48]:
#%% read in model parameter data

FUEL = pd.read_csv(Path('..','..',f'{folder_in}', 'data_csv', 'FUEL.csv'))
TECH = pd.read_csv(Path('..','..',f'{folder_in}', 'data_csv', 'TECHNOLOGY.csv'))

## energy fuels
energy_fuel = FUEL.loc[(FUEL.VALUE.str[0:2] == 'C1')]
PrimEN_fuel = (energy_fuel.loc[(energy_fuel.VALUE.str[3:4] == 'P')|(energy_fuel.VALUE.str[3:4] == 'R')]).VALUE.unique()
SecEN_fuel = (energy_fuel.loc[(energy_fuel.VALUE.str[3:4] == 'S')]).VALUE.unique()
#print(SecEN_fuel)
FinalEN_fuel = (energy_fuel.loc[(energy_fuel.VALUE.str[3:4] == 'F')]).VALUE.unique()
#print(FinalEN_fuel)
## energy techs
energy_tech = TECH.loc[(TECH.VALUE.str[0:2] == 'C1')]
ResEN_tech = (energy_tech.loc[(energy_tech.VALUE.str[6:7] == 'I')]).VALUE.unique()
RE_tech = (energy_tech.loc[(energy_tech.VALUE.str[2:4] == 'SO')|(energy_tech.VALUE.str[2:4] == 'WD')|(energy_tech.VALUE.str[2:4] == 'HY')|(energy_tech.VALUE.str[2:4] == 'GO')|(energy_tech.VALUE.str[2:4] == 'OC')]).VALUE.unique()
PrimEN_tech = (energy_tech.loc[(energy_tech.VALUE.str[6:7] == 'P')|(energy_tech.VALUE.str[6:7] == '0')]).VALUE.unique()
SecEN_tech = (energy_tech.loc[(energy_tech.VALUE.str[6:7] == 'T')]).VALUE.unique()
#print(SecEN_tech)
FinalEN_tech = (energy_tech.loc[(energy_tech.VALUE.str[6:7] == 'F')]).VALUE.unique()
#print(FinalEN_tech)

## land and food fuels
land_fuel = (FUEL.loc[(FUEL.VALUE.str[0:1] == 'L')]).VALUE.unique()
food_fuel = (FUEL.loc[(FUEL.VALUE.str[0:1] == 'M')|(FUEL.VALUE.str[0:1] == 'V')]).VALUE.unique()
## land and food techs
land_tech = (TECH.loc[(TECH.VALUE.str[0:1] == 'L')]).VALUE.unique()
food_tech = (TECH.loc[(TECH.VALUE.str[0:1] == 'M')|(TECH.VALUE.str[0:1] == 'V')]).VALUE.unique()

## materials fuels
material_fuel = (FUEL.loc[(FUEL.VALUE.str.len() == 3)|(FUEL.VALUE.str.len() == 4)]).VALUE.unique()
print(material_fuel)
## materials techs
industry_tech = (TECH.loc[(TECH.VALUE.str[-5:] == 'PLANT')]).VALUE.unique()
print(industry_tech)
material_tech = (TECH.loc[(TECH.VALUE.str[-4:] == 'MINE')|(TECH.VALUE.str[-7:] == 'RECYCLE')]).VALUE.unique()
print(material_tech)

## emissions
EMISSION = pd.read_csv(Path('..','..',f'{folder_in}','data_csv', 'EMISSION.csv'))
emissions = (EMISSION.loc[(EMISSION.VALUE.str.len() == 5)]).VALUE.unique()

## timeframe / modelling period
YEAR = pd.read_csv(Path('..','..',f'{folder_in}','data_csv', 'YEAR.csv'))
YEAR = YEAR.iloc[:-10]
#print(YEAR)

# parameter_file = glob.glob1(folder_in, '*parameters*')
# parameters_GLUCOSE = pd.read_csv(Path('..','..',f'{folder_in}',f'{parameter_file}'))
# col = parameters_GLUCOSE['indexes']

['ALU' 'AMM' 'CEM' 'FERT' 'LAND' 'MFOO' 'PAP' 'PET' 'STE' 'VFOO' 'XALU'
 'XCEM' 'XPHO' 'XPOT' 'XSTE']
['ALUPLANT' 'ALURCPLANT' 'CEMPLANT' 'CEMCSPLANT' 'FERTPLANT' 'PAPELPLANT'
 'PAPPLANT' 'PETAPLANT' 'PETBELPLANT' 'PETBPLANT' 'STEHGPLANT' 'STEPLANT']
['ALURECYCLE' 'XALUMINE' 'XCEMMINE' 'XPHOMINE' 'XPOTMINE' 'XSTEMINE']


In [4]:
# Function for calculating yearly technology penetration (incremental share of electrical capacity)
def percentage_change(col1,col2):
    return ((col2 - col1) / col1) * 100

In [5]:
# #%% read in value of uncertainty inputs in 2050 - see parameters.csv file under config/ folder - only for RDM analysis

# inputs_GLUCOSE = pd.DataFrame(index=replications)
# yr = 2050


# for variable in (parameters_GLUCOSE['name']):
# #path = 'results/0/model_{run}/data/AccumulatedAnnualDemand.csv'
#     csv_file_data = Path('..', '..', f'{folder_in}','data_csv', f'{variable}.csv')
#     #csv_file_data2 = Path('results', '0', f'model_{run}', 'data', 'SpecifiedAnnualDemand.csv')
#     if not os.path.exists(csv_file_data):
#         pass
#     else:
#         data=pd.read_csv(csv_file_data)
#         parameters = parameters_GLUCOSE.loc[parameters_GLUCOSE['name']==variable]
#         for param in parameters['indexes']:
#             if variable[-6:] == 'Demand':
#                 fuel = param[6:]
#                 selection = data.loc[(data["FUEL"]==fuel)&(data["YEAR"]==yr)]
#                 inputs_GLUCOSE.loc[run,(''.join(c for c in variable if c.isupper())+','+fuel)]=selection.VALUE.sum() 
#             if (variable == 'InputActivityRatio')|(variable == 'OutputActivityRatio'):
#                 technology = param.split(",")[1].strip()
#                 fuel = param.split(",")[2].strip()
#                 ModOp = param.split(",")[3].strip()
#                 selection = data.loc[(data["TECHNOLOGY"]==technology)&(data["FUEL"]==fuel)&(data["MODE_OF_OPERATION"]==ModOp)&(data["YEAR"]==yr)]
#                 inputs_GLUCOSE.loc[run,(''.join(c for c in variable if c.isupper())+','+technology+','+fuel)]=selection.VALUE.sum() 
#             if (variable == 'TotalAnnualMaxCapacityInvestment')|(variable == 'TotalTechnologyAnnualActivityUpperLimit'):
#                 technology = param.split(",")[1].strip()
#                 selection = data.loc[(data["TECHNOLOGY"]==technology)&(data["YEAR"]==yr)]
#                 inputs_GLUCOSE.loc[run,(''.join(c for c in variable if c.isupper())+','+technology)]=selection.VALUE.sum() 
#             if variable == 'AnnualEmissionLimit':
#                 emission = param.split(",")[1].strip()
#                 selection = data.loc[(data["EMISSION"]==emission)&(data["YEAR"]==yr)]
#                 inputs_GLUCOSE.loc[run,(''.join(c for c in variable if c.isupper())+','+emission)]=selection.VALUE.sum() 
#             if variable == 'EmissionActivityRatio':
#                 technology = param.split(",")[1].strip()
#                 emission = param.split(",")[2].strip()
#                 selection = data.loc[(data["TECHNOLOGY"]==technology)&(data["EMISSION"]==emission)&(data["YEAR"]==yr)]
#                 inputs_GLUCOSE.loc[run,(''.join(c for c in variable if c.isupper())+','+technology+','+emission)]=selection.VALUE.sum() 
#             if variable == 'EmissionsPenalty':
#                 #technology = param.split(",")[1].strip()
#                 emission = param.split(",")[1].strip()
#                 selection = data.loc[(data["EMISSION"]==emission)&(data["YEAR"]==yr)]
#                 inputs_GLUCOSE.loc[run,(''.join(c for c in variable if c.isupper())+','+emission)]=selection.VALUE.sum() 


In [6]:
# # save uncertainty inputs into file
# output = 'GLUCOSE_DataInputs'

# #fp = open(f'{output}'+'.p', 'wb')
# my_path = Path('..', '..', f'{folder_out}', 'results_processed', f'{output}'+'.p') 
# with my_path.open('wb') as fp:
#     pickle.dump(inputs_GLUCOSE, fp)

In [7]:
CO2EQ_GLUCOSE = pd.Series(index=YEAR.VALUE, dtype='float64') 
ccs_em = pd.Series(index=YEAR.VALUE, dtype='float64')  
WATER_GLUCOSE = pd.Series(index=YEAR.VALUE, dtype='float64')  
LAi_GLUC = pd.Series(index=YEAR.VALUE, dtype='float64')  
LF_GLUCOSE = pd.Series(index=YEAR.VALUE, dtype='float64')  
NU_prod = pd.Series(index=YEAR.VALUE, dtype='float64')  
NU_cap = pd.Series(index=YEAR.VALUE, dtype='float64')  
BM_use = pd.Series(index=YEAR.VALUE, dtype='float64')  

In [8]:
#%% processing results - read in selected results for year 2050
results_file1 = Path('..', '..', f'{folder_out}', 'results_csv', 'TotalAnnualTechnologyActivityByMode.csv')
results_file2 = Path('..', '..', f'{folder_out}', 'results_csv', 'AnnualEmissions.csv')
results_file3 = Path('..', '..', f'{folder_out}', 'results_csv', 'AnnualTechnologyEmission.csv')
results_file4 = Path('..', '..', f'{folder_out}', 'results_csv', 'ProductionByTechnologyAnnual.csv')
results_file5 = Path('..', '..', f'{folder_out}', 'results_csv', 'TotalCapacityAnnual.csv')
results_file6 = Path('..', '..', f'{folder_out}', 'results_csv', 'UseByTechnology.csv')

if not os.path.exists(results_file1):
    pass
else:
    TotalAnnualTechnologyActivityByMode=pd.read_csv(results_file1)
    AnnualEmissions = pd.read_csv(results_file2)
    AnnualTechnologyEmission = pd.read_csv(results_file3)
    ProductionByTechnologyAnnual = pd.read_csv(results_file4)
    TotalCapacityAnnual = pd.read_csv(results_file5)
    UseByTechnology = pd.read_csv(results_file6)        
    for year in YEAR.VALUE:
        for tech in land_tech:
            if tech == 'LF':
                land_temp = TotalAnnualTechnologyActivityByMode[(TotalAnnualTechnologyActivityByMode['TECHNOLOGY'] == tech)&(TotalAnnualTechnologyActivityByMode['YEAR'] == year)&(TotalAnnualTechnologyActivityByMode['MODE_OF_OPERATION'] == 1)]
                LF_GLUCOSE.loc[year] = land_temp['VALUE'].sum()
            if tech == 'LA1_i':
                land_temp = TotalAnnualTechnologyActivityByMode[(TotalAnnualTechnologyActivityByMode['TECHNOLOGY'] == tech)&(TotalAnnualTechnologyActivityByMode['YEAR'] == year)&(TotalAnnualTechnologyActivityByMode['MODE_OF_OPERATION'] == 1)]
                LAi_GLUC.loc[year] = land_temp['VALUE'].sum()
            for tech in PrimEN_tech:
                if tech[2:4]=='NU':
                    production = ProductionByTechnologyAnnual[(ProductionByTechnologyAnnual['TECHNOLOGY']==tech)&(ProductionByTechnologyAnnual['YEAR']== year)]
                    capacity = TotalCapacityAnnual[(TotalCapacityAnnual['TECHNOLOGY']==tech)&(TotalCapacityAnnual['YEAR']== year)]
                    NU_prod.loc[year] = production['VALUE'].sum()
                    NU_cap.loc[year] = capacity['VALUE'].sum()
        use = UseByTechnology.loc[(UseByTechnology['TECHNOLOGY'].str[2:4]=='BM')&(UseByTechnology['FUEL']=='C1_P_BIOW')&(UseByTechnology['YEAR']==year)]
        #UseByTechnology[(UseByTechnology['FUEL']==fuel)&(UseByTechnology['TECHNOLOGY']==tech)&(UseByTechnology['YEAR']==year)]
        BM_use.loc[year] = use['VALUE'].sum()
        for ems in emissions:
            if ems == 'CO2EQ':
                em = AnnualEmissions[(AnnualEmissions['EMISSION'] == ems)&(AnnualEmissions['YEAR'] == year)]
                AnnualTechnologyEmission_ccs = AnnualTechnologyEmission.loc[(AnnualTechnologyEmission['TECHNOLOGY'].str[-2:] == 'CS')|(AnnualTechnologyEmission['TECHNOLOGY'].str[-7:-5] == 'CS')]
                AnnualTechnologyEmission_ccs_em = AnnualTechnologyEmission_ccs[(AnnualTechnologyEmission_ccs['EMISSION'] == ems)&(AnnualTechnologyEmission_ccs['YEAR'] == year)]
                CO2EQ_GLUCOSE.loc[year] = em.VALUE.sum()# em['VALUE'].values
                if not AnnualTechnologyEmission_ccs_em.empty:
                    AnnualTechnologyEmission_ccs = (AnnualTechnologyEmission.loc[(AnnualTechnologyEmission['TECHNOLOGY'].str[-2:] == 'CS')|(AnnualTechnologyEmission['TECHNOLOGY'].str[-7:-5] == 'CS')])[(AnnualTechnologyEmission['EMISSION'] == ems)&(AnnualTechnologyEmission['YEAR'] == year)]
                    ccs_em.loc[year] = AnnualTechnologyEmission_ccs_em.VALUE.sum()
            if ems == 'WATER':
                em = AnnualEmissions[(AnnualEmissions['EMISSION'] == ems)&(AnnualEmissions['YEAR'] == year)]
                WATER_GLUCOSE.loc[year] = em.VALUE.sum()

/var/folders/g0/db564x8n6j14t8scrcvbrmsc0000gn/T/ipykernel_33928/3559684680.py:42: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  AnnualTechnologyEmission_ccs = (AnnualTechnologyEmission.loc[(AnnualTechnologyEmission['TECHNOLOGY'].str[-2:] == 'CS')|(AnnualTechnologyEmission['TECHNOLOGY'].str[-7:-5] == 'CS')])[(AnnualTechnologyEmission['EMISSION'] == ems)&(AnnualTechnologyEmission['YEAR'] == year)]


In [9]:
#%% processing results - read in selected results for year 2050

# CO2EQ_GLUCOSE = pd.DataFrame(columns=YEAR.VALUE).fillna(0)
# ccs_em = pd.DataFrame(columns=YEAR.VALUE).fillna(0)
# WATER_GLUCOSE = pd.DataFrame(columns=YEAR.VALUE).fillna(0)
# LAi_GLUC = pd.DataFrame(columns=YEAR.VALUE).fillna(0)
# LF_GLUCOSE = pd.DataFrame(columns=YEAR.VALUE).fillna(0)
# NU_prod = pd.DataFrame(columns=YEAR.VALUE).fillna(0)
# NU_cap = pd.DataFrame(columns=YEAR.VALUE).fillna(0)
# BM_use = pd.DataFrame(columns=YEAR.VALUE).fillna(0)


# results_file1 = Path('..', '..', f'{folder_out}', 'results_csv', 'TotalAnnualTechnologyActivityByMode.csv')
# results_file2 = Path('..', '..', f'{folder_out}', 'results_csv', 'AnnualEmissions.csv')
# results_file3 = Path('..', '..', f'{folder_out}', 'results_csv', 'AnnualTechnologyEmission.csv')
# results_file4 = Path('..', '..', f'{folder_out}', 'results_csv', 'ProductionByTechnologyAnnual.csv')
# results_file5 = Path('..', '..', f'{folder_out}', 'results_csv', 'TotalCapacityAnnual.csv')
# results_file6 = Path('..', '..', f'{folder_out}', 'results_csv', 'UseByTechnology.csv')
# if not os.path.exists(results_file1):
#     pass
# else:
#     TotalAnnualTechnologyActivityByMode=pd.read_csv(results_file1)
#     AnnualEmissions = pd.read_csv(results_file2)
#     AnnualTechnologyEmission = pd.read_csv(results_file3)
#     ProductionByTechnologyAnnual = pd.read_csv(results_file4)
#     TotalCapacityAnnual = pd.read_csv(results_file5)
#     UseByTechnology = pd.read_csv(results_file6)        
#     for year in YEAR.VALUE:
#         for tech in land_tech:
#             if tech == 'LF':
#                 land_temp = TotalAnnualTechnologyActivityByMode[(TotalAnnualTechnologyActivityByMode['TECHNOLOGY'] == tech)&(TotalAnnualTechnologyActivityByMode['YEAR'] == year)&(TotalAnnualTechnologyActivityByMode['MODE_OF_OPERATION'] == 1)]
#                 LF_GLUCOSE.loc[0,year] = land_temp['VALUE'].sum()
#             if tech == 'LA1_i':
#                 land_temp = TotalAnnualTechnologyActivityByMode[(TotalAnnualTechnologyActivityByMode['TECHNOLOGY'] == tech)&(TotalAnnualTechnologyActivityByMode['YEAR'] == year)&(TotalAnnualTechnologyActivityByMode['MODE_OF_OPERATION'] == 1)]
#                 LAi_GLUC.loc[0,year] = land_temp['VALUE'].sum()
#             for tech in PrimEN_tech:
#                 if tech[2:4]=='NU':
#                     production = ProductionByTechnologyAnnual[(ProductionByTechnologyAnnual['TECHNOLOGY']==tech)&(ProductionByTechnologyAnnual['YEAR']== year)]
#                     capacity = TotalCapacityAnnual[(TotalCapacityAnnual['TECHNOLOGY']==tech)&(TotalCapacityAnnual['YEAR']== year)]
#                     NU_prod.loc[0,year] = production['VALUE'].sum()
#                     NU_cap.loc[0,year] = capacity['VALUE'].sum()
#         use = UseByTechnology.loc[(UseByTechnology['TECHNOLOGY'].str[2:4]=='BM')&(UseByTechnology['FUEL']=='C1_P_BIOW')&(UseByTechnology['YEAR']==year)]
#         #UseByTechnology[(UseByTechnology['FUEL']==fuel)&(UseByTechnology['TECHNOLOGY']==tech)&(UseByTechnology['YEAR']==year)]
#         BM_use.loc[0,year] = use['VALUE'].sum()
#         for ems in emissions:
#             if ems == 'CO2EQ':
#                 em = AnnualEmissions[(AnnualEmissions['EMISSION'] == ems)&(AnnualEmissions['YEAR'] == year)]
#                 AnnualTechnologyEmission_ccs = AnnualTechnologyEmission.loc[(AnnualTechnologyEmission['TECHNOLOGY'].str[-2:] == 'CS')|(AnnualTechnologyEmission['TECHNOLOGY'].str[-7:-5] == 'CS')]
#                 AnnualTechnologyEmission_ccs_em = AnnualTechnologyEmission_ccs[(AnnualTechnologyEmission_ccs['EMISSION'] == ems)&(AnnualTechnologyEmission_ccs['YEAR'] == year)]
#                 CO2EQ_GLUCOSE.loc[0,year] = em.VALUE.sum()# em['VALUE'].values
#                 if not AnnualTechnologyEmission_ccs_em.empty:
#                     AnnualTechnologyEmission_ccs = (AnnualTechnologyEmission.loc[(AnnualTechnologyEmission['TECHNOLOGY'].str[-2:] == 'CS')])[(AnnualTechnologyEmission['EMISSION'] == ems)&(AnnualTechnologyEmission['YEAR'] == year)]
#                     ccs_em.loc[0,year] = AnnualTechnologyEmission_ccs_em.VALUE.sum()
#             if ems == 'WATER':
#                 em = AnnualEmissions[(AnnualEmissions['EMISSION'] == ems)&(AnnualEmissions['YEAR'] == year)]
#                 WATER_GLUCOSE.loc[0,year] = em.VALUE.sum()

In [10]:
#%% read in GLUCOSE results: ProductionByTechnologyAnnual - Primary Energy
PrimEn = {}
RE_tech = pd.Series(RE_tech).str[2:4].unique()
#ResEN_tech = pd.Series(ResEN_tech).str[2:4].unique()
ResEN_tech = np.append(ResEN_tech, 'BM')


PrimEn = pd.DataFrame(index=YEAR.VALUE, columns=(list(ResEN_tech)+list(RE_tech))).fillna(0)
results_file1 = Path('..', '..', f'{folder_out}', 'results_csv', 'ProductionByTechnologyAnnual.csv')
results_file2 = Path('..', '..', f'{folder_out}', 'results_csv', 'UseByTechnology.csv')
if not os.path.exists(results_file1):
    pass
else:
    ProductionByTechnologyAnnual=pd.read_csv(results_file1)
    UseByTechnology=pd.read_csv(results_file2)
    for year in YEAR.VALUE:
        for tech in ResEN_tech:
            PrimEn_temp = ProductionByTechnologyAnnual[(ProductionByTechnologyAnnual['TECHNOLOGY'] == tech)&(ProductionByTechnologyAnnual['YEAR'] == year)]
            PrimEn.loc[year,tech] = PrimEn_temp['VALUE'].sum()
        Use_temp = UseByTechnology.loc[(UseByTechnology['TECHNOLOGY'].str[2:4]=='BM')&(UseByTechnology['FUEL']=='C1_P_BIOW')&(UseByTechnology['YEAR']==year)]
        PrimEn.loc[year, 'BM']+= Use_temp['VALUE'].sum()
        for tech in RE_tech:
            PrimREn_temp = ProductionByTechnologyAnnual.loc[(ProductionByTechnologyAnnual['TECHNOLOGY'].str[2:4] == tech)&(ProductionByTechnologyAnnual['YEAR'] == year)]
            PrimEn.loc[year, tech] = PrimREn_temp['VALUE'].sum()


In [11]:
# RE_tech = pd.Series(RE_tech).str[2:4].unique()
# print(RE_tech)
# #ResEN_tech = pd.Series(ResEN_tech).str[2:4].unique()
# #ResEN_tech = np.append(ResEN_tech)#, 'BM')
# print(ResEN_tech)

# PrimEn= pd.DataFrame(columns=['RUN', 'TECHNOLOGY', 'YEAR', 'VALUE'])

In [12]:
#%% read in GLUCOSE results: ProductionByTechnologyAnnual - Primary Energy
#PrimEn = {}
#RE_tech = pd.Series(RE_tech).str[2:4].unique()
#ResEN_tech = pd.Series(ResEN_tech).str[2:4].unique()
#ResEN_tech = np.append(ResEN_tech)#, 'BM')
#print(ResEN_tech)

# #PrimEn= pd.DataFrame(columns=['RUN', 'TECHNOLOGY', 'YEAR', 'VALUE'])
# for run in replications:
#     results_file1 = Path(f'{folder}', 'results', '0', f'{run}', 'ProductionByTechnologyAnnual.csv')
#     results_file2 = Path(f'{folder}', 'results', '0', f'{run}', 'UseByTechnology.csv')
#     if not os.path.exists(results_file1):
#         pass
#     else:
#         ProductionByTechnologyAnnual=pd.read_csv(results_file1)
#         UseByTechnology=pd.read_csv(results_file2)
#         for tech in ResEN_tech:
#             PrimEn_temp = ProductionByTechnologyAnnual[(ProductionByTechnologyAnnual['TECHNOLOGY'] == tech)]
#             PrimEn_temp=PrimEn_temp.drop(['REGION','FUEL'], axis=1)
#             PrimEn_temp['RUN'] = run
#             print(PrimEn_temp)
#             PrimEn = pd.concat([PrimEn, PrimEn_temp], axis=0, ignore_index=True)
#         #print(PrimEn)
#         for tech in RE_tech:
#             PrimREn_temp = ProductionByTechnologyAnnual.loc[(ProductionByTechnologyAnnual['TECHNOLOGY'].str[2:4] == tech)]
#             PrimREn_temp['VALUE']=PrimREn_temp.groupby(['YEAR'])['VALUE'].transform(sum)
#             PrimREn_temp = PrimREn_temp.drop(['REGION','FUEL'], axis=1)
#             PrimREn_temp['RUN'] = run
#             PrimREn_temp['TECHNOLOGY'] = tech
#             print(PrimREn_temp)
#             PrimEn = pd.concat([PrimEn, PrimREn_temp], axis=0, ignore_index=True)
#         print(PrimEn)
        # Use_temp = UseByTechnology.loc[(UseByTechnology['TECHNOLOGY'].str[2:4]=='BM')&(UseByTechnology['FUEL']=='C1_P_BIOW')]
        # #Use_temp=Use_temp.drop(['REGION', 'TIMESLICE'], axis=1)
        # Use_temp['VALUE']=Use_temp.groupby(['YEAR'])['VALUE'].transform(sum)
        # #print(Use_temp)
        # Use_temp=Use_temp.drop(['REGION','TIMESLICE','FUEL'], axis=1)
        # Use_temp['RUN'] = run
        # Use_temp['TECHNOLOGY'] = 'BM'
        # print(Use_temp)
        # PrimEn = pd.concat([PrimEn, Use_temp], axis=0, ignore_index=True)
        #     
        # for year in Use_temp.YEAR.unique():
        #     value = Use_temp.loc[Use_temp['YEAR'] == year, 'VALUE'].sum()
        #     Use_temp.drop(['REGION','TIMESLICE','FUEL'], axis=1)
        #     Use_temp['RUN'] = run
        #     Use_temp['TECHNOLOGY'] = 'BM'
        #     Use_temp['VALUE'] = value
        #     #print(Use_temp)
        #     PrimEn = pd.concat([PrimEn, Use_temp], axis=0, ignore_index=True)

        # for tech in RE_tech:
        #     PrimREn_temp = ProductionByTechnologyAnnual.loc[(ProductionByTechnologyAnnual['TECHNOLOGY'].str[2:4] == tech)]
        #     PrimREn_temp.drop(['REGION','FUEL'], axis=1)
        #     PrimREn_temp['RUN'] = run
        #     PrimREn_temp['TECHNOLOGY'] = tech


In [13]:
# save results ProductionByTechnologyAnnual - Primary Energy into file
output = 'GLUCOSE_PrimaryEnergy'

#fp = open(f'{output}'+'.p', 'wb')
my_path = Path('..', '..', f'{folder_out}', 'results_processed', f'{output}'+'.p') 
with my_path.open('wb') as fp:
    pickle.dump(PrimEn, fp)

In [14]:
#%% read in GLUCOSE results: TotalCapacityAnnual - Electrical Capacity
Capacity_elc = {}
#RE_tech = pd.Series(RE_tech).str[2:4].unique()
PrimEN_tech_abb = (pd.Series(PrimEN_tech).str[2:4].unique())
PrimEN_tech = np.append(PrimEN_tech, 'BM')


Capacity_elc = pd.DataFrame(index=YEAR.VALUE, columns=(list(PrimEN_tech_abb))).fillna(0)
results_file1 = Path('..', '..', f'{folder_out}', 'results_csv', 'TotalCapacityAnnual.csv')
results_file2 = Path('..', '..', f'{folder_in}', 'data_csv', 'OutputActivityRatio.csv')
if not os.path.exists(results_file1)|os.path.exists(results_file2):
    pass
else:
    TotalCapacityAnnual=pd.read_csv(results_file1)
    OutputActivityRatio=pd.read_csv(results_file2)
    for tech in PrimEN_tech:
        if tech[-2:] != 'CS':
            for year in YEAR.VALUE:
                #for tech in PrimEN_tech:
                OAR = (OutputActivityRatio[(OutputActivityRatio['TECHNOLOGY'] == tech)&(OutputActivityRatio['YEAR'] == year)&(OutputActivityRatio['MODE_OF_OPERATION'] == 1)]).FUEL
                for oar in OAR:
                    if oar == 'C1_S_ELC':
                        Capacity_elc_temp = TotalCapacityAnnual[(TotalCapacityAnnual['TECHNOLOGY'] == tech)&(TotalCapacityAnnual['YEAR'] == year)]
                        tech_abb = tech[2:4]
                        Capacity_elc.loc[year,tech_abb] += Capacity_elc_temp['VALUE'].sum()
        else:
            ccs_abb = tech[2:4]+tech[-2:]
            Capacity_elc[ccs_abb]=0
            for year in YEAR.VALUE:
                #for tech in PrimEN_tech:
                OAR = (OutputActivityRatio[(OutputActivityRatio['TECHNOLOGY'] == tech)&(OutputActivityRatio['YEAR'] == year)&(OutputActivityRatio['MODE_OF_OPERATION'] == 1)]).FUEL
                for oar in OAR:
                    if oar == 'C1_S_ELC':
                        Capacity_elc_temp = TotalCapacityAnnual[(TotalCapacityAnnual['TECHNOLOGY'] == tech)&(TotalCapacityAnnual['YEAR'] == year)]
                        Capacity_elc.loc[year,ccs_abb] += Capacity_elc_temp['VALUE'].sum()

        # Use_temp = UseByTechnology.loc[(UseByTechnology['TECHNOLOGY'].str[2:4]=='BM')&(UseByTechnology['FUEL']=='C1_P_BIOW')&(UseByTechnology['YEAR']==year)]
        # PrimEn[run].loc[year, 'BM']+= Use_temp['VALUE'].sum()
        # for tech in RE_tech:
        #     TPES_temp = ProductionByTechnologyAnnual.loc[(ProductionByTechnologyAnnual['TECHNOLOGY'].str[2:4] == tech)&(ProductionByTechnologyAnnual['YEAR'] == year)]
        #     TPES[run].loc[year, tech] = TPES_temp['VALUE'].sum()

In [15]:
# save results TotalCapacityAnnual - Electrical Capacity into file
output = 'GLUCOSE_ElectricalCapacity'

#fp = open(f'{output}'+'.p', 'wb')
my_path = Path('..', '..', f'{folder_out}', 'results_processed', f'{output}'+'.p') 
with my_path.open('wb') as fp:
    pickle.dump(Capacity_elc, fp)

In [16]:
#%% read in GLUCOSE results: TotalAnnualTechnologyActivityByMode - Land Use
LandResources = {}
land_tech = ['LA1', 'LA1_i', 'LA2', 'LF1', 'LF2', 'LO']
# for tech in land_tech:
#     LandResources[run]=pd.DataFrame(index=YEAR.VALUE, columns=land_tech).fillna(0)


LandResources=pd.DataFrame(index=YEAR.VALUE, columns=(list(land_tech))).fillna(0)
results_file1 = Path('..', '..', f'{folder_out}', 'results_csv', 'TotalAnnualTechnologyActivityByMode.csv')
if not os.path.exists(results_file1):
    pass
else:
    TotalAnnualTechnologyActivityByMode=pd.read_csv(results_file1)
    for year in YEAR.VALUE:
        for tech in land_tech:
            Land_temp = TotalAnnualTechnologyActivityByMode[(TotalAnnualTechnologyActivityByMode['TECHNOLOGY'] == tech)&(TotalAnnualTechnologyActivityByMode['YEAR'] == year)&(TotalAnnualTechnologyActivityByMode['MODE_OF_OPERATION'] == 1)]
            LandResources.loc[year, tech] = Land_temp['VALUE'].sum()

In [17]:
# save results TotalAnnualTechnologyActivityByMode - Land Use into file
output = 'GLUCOSE_LandUse'

#fp = open(f'{output}'+'.p', 'wb')
my_path = Path('..', '..', f'{folder_out}', 'results_processed', f'{output}'+'.p') 
with my_path.open('wb') as fp:
    pickle.dump(LandResources, fp)

In [49]:
#%% read in GLUCOSE results: ProductionByTechnologyAnnual - Industry
IndustryProd = {}

IndustryProd=pd.DataFrame(index=YEAR.VALUE, columns=(list(industry_tech))).fillna(0)
results_file1 = Path('..', '..', f'{folder_out}', 'results_csv', 'ProductionByTechnologyAnnual.csv')
if not os.path.exists(results_file1):
    pass
else:
    ProductionByTechnologyAnnual=pd.read_csv(results_file1)
    for year in YEAR.VALUE:
        for tech in industry_tech:
            for fuel in material_fuel:
                Industry_temp = ProductionByTechnologyAnnual[(ProductionByTechnologyAnnual['TECHNOLOGY'] == tech)&(ProductionByTechnologyAnnual['YEAR'] == year)&(ProductionByTechnologyAnnual['FUEL'] == fuel)]
                IndustryProd.loc[year, tech] += Industry_temp['VALUE'].sum()

In [50]:
# save results ProductionByTechnologyAnnual - Industry into file
output = 'GLUCOSE_IndustryProduction'

#fp = open(f'{output}'+'.p', 'wb')
my_path = Path('..', '..', f'{folder_out}', 'results_processed', f'{output}'+'.p') 
with my_path.open('wb') as fp:
    pickle.dump(IndustryProd, fp)

In [18]:
#%% read in GLUCOSE results: ProductionByTechnologyAnnual - Primary Energy, RENEWABLES
# RE penetration 
RE_prod = pd.DataFrame(columns=YEAR.VALUE).fillna(0)

PrimEn_RE = PrimEn.iloc[: , -6:].sum(axis=1)
RE_prod = PrimEn_RE.transpose()

In [19]:
# save GLUCOSE results: ProductionByTechnologyAnnual - Primary Energy, RENEWABLES into file
output = 'GLUCOSE_PrimaryEnergy_RE'

#fp = open(f'{output}'+'.p', 'wb')
my_path = Path('..', '..', f'{folder_out}', 'results_processed', f'{output}'+'.p') 
with my_path.open('wb') as fp:
    pickle.dump(RE_prod, fp)

In [20]:
#%% read in GLUCOSE results: TotalCapacityAnnual - Electrical Capacity, RENEWABLES
# RE penetration 

#RES = pd.Series(pd.Series(RE_tech).str[2:4].unique())
RES = np.append(RE_tech, 'BM')
RE_cap = pd.DataFrame(columns=YEAR.VALUE).fillna(0)

ElcCap_RE = pd.Series(index=YEAR.VALUE, dtype=float).fillna(0)
for t in RES:
    ElcCap_RE += Capacity_elc[t]
RE_cap = ElcCap_RE.transpose()


In [21]:
# save GLUCOSE results: TotalCapacityAnnual - Electrical Capacity, RENEWABLES into file
output = 'GLUCOSE_ElectricalCapacity_RE'

#fp = open(f'{output}'+'.p', 'wb')
my_path = Path('..', '..', f'{folder_out}', 'results_processed', f'{output}'+'.p') 
with my_path.open('wb') as fp:
    pickle.dump(RE_cap, fp)

In [22]:
# calculate technology penetration rate: TotalCapacityAnnual - Electrical Capacity, RENEWABLES 
RE_cap_diff = pd.Series(index=range(2020,2051))
for c in RE_cap_diff.index:
    RE_cap_diff[c]= RE_cap[c] - RE_cap[c-1]

/var/folders/g0/db564x8n6j14t8scrcvbrmsc0000gn/T/ipykernel_33928/2870987108.py:2: FutureWarning: The default dtype for empty Series will be 'object' instead of 'float64' in a future version. Specify a dtype explicitly to silence this warning.
  RE_cap_diff = pd.Series(index=range(2020,2051))


In [23]:
# save technology penetration rate: TotalCapacityAnnual - Electrical Capacity, RENEWABLE
output = 'GLUCOSE_ElectricalCapacity_RE_diff'

#fp = open(f'{output}'+'.p', 'wb')
my_path = Path('..', '..', f'{folder_out}', 'results_processed', f'{output}'+'.p') 
with my_path.open('wb') as fp:
    pickle.dump(RE_cap_diff, fp)

In [24]:
#%% read in GLUCOSE results: ProductionByTechnologyAnnual - Primary Energy, NUCLEAR
# nuclear penetration
NU_prod = pd.Series(index=YEAR.VALUE, dtype='float64') #.fillna(0)

PrimEn_techs = pd.Series(PrimEn.columns) 
NU_tech = PrimEn_techs[PrimEn_techs.str[2:4]=='NU']
PrimEn_NU = PrimEn[NU_tech]
NU_prod = PrimEn_NU

In [25]:
# save GLUCOSE results: ProductionByTechnologyAnnual - Primary Energy, NUCLEAR into file
output = 'GLUCOSE_PrimaryEnergy_NU'

#fp = open(f'{output}'+'.p', 'wb')
my_path = Path('..', '..', f'{folder_out}', 'results_processed', f'{output}'+'.p') 
with my_path.open('wb') as fp:
    pickle.dump(NU_prod, fp)

In [26]:
#%% read in GLUCOSE results: TotalCapacityAnnual - Electrical Capacity, NUCLEAR
# nuclear penetration
NU_cap = pd.Series(index=YEAR.VALUE, dtype='float64')#.fillna(0)

NU_tech = PrimEn_techs[PrimEn_techs.str[2:4]=='NU']
#CapElc_NU = Capacity_elc[run][NU_tech.str[2:4]]
NU_cap = Capacity_elc[NU_tech.str[2:4]].transpose()

In [27]:
# save GLUCOSE results: TotalCapacityAnnual - Electrical Capacity, NUCLEAR
output = 'GLUCOSE_ElectricalCapacity_NU'

#fp = open(f'{output}'+'.p', 'wb')
my_path = Path('..', '..', f'{folder_out}', 'results_processed', f'{output}'+'.p') 
with my_path.open('wb') as fp:
    pickle.dump(NU_cap, fp)

In [28]:
# calculate technology penetration rate: TotalCapacityAnnual - Electrical Capacity, NUCLEAR 
NU_cap_diff = pd.Series(index=range(2020,2051), dtype='float64')
for c in NU_cap_diff.index:
    NU_cap_diff[c]=NU_cap[c]-NU_cap[c-1]

NU_cap_diff = NU_cap_diff.T

In [29]:
# save technology penetration rate: TotalCapacityAnnual - Electrical Capacity, NUCLEAR 
output = 'GLUCOSE_ElectricalCapacity_NU_diff'

#fp = open(f'{output}'+'.p', 'wb')
my_path = Path('..', '..', f'{folder_out}', 'results_processed', f'{output}'+'.p') 
with my_path.open('wb') as fp:
    pickle.dump(NU_cap_diff, fp)

In [30]:
#%% read in GLUCOSE results: TotalCapacityAnnual - Electrical Capacity, CCS
# CCS penetration
CCS_ElcCap = pd.Series(index=YEAR.VALUE, dtype='float64')

Capacity_elc_techs = pd.Series(Capacity_elc.columns) 
CCS_tech = Capacity_elc_techs[Capacity_elc_techs.str[-2:]=='CS']
Cap_CCS = Capacity_elc[CCS_tech].sum(axis=1)
CCS_ElcCap = Cap_CCS.T


In [31]:
# save GLUCOSE results: TotalCapacityAnnual - Electrical Capacity, CCS into file
output = 'GLUCOSE_ElectricalCapacity_CCS'

#fp = open(f'{output}'+'.p', 'wb')
my_path = Path('..', '..', f'{folder_out}', 'results_processed', f'{output}'+'.p') 
with my_path.open('wb') as fp:
    pickle.dump(CCS_ElcCap, fp)

In [32]:
# calculate technology penetration rate: TotalCapacityAnnual - Electrical Capacity, CCS
CCS_ElcCap_diff = pd.Series(index=range(2020,2051))
for c in CCS_ElcCap_diff.index:
    CCS_ElcCap_diff[c]=CCS_ElcCap[c]-CCS_ElcCap[c-1]

# save GLUCOSE results: TotalCapacityAnnual - Electrical Capacity, CCS into file
output = 'GLUCOSE_ElectricalCapacity_CCS_diff'

#fp = open(f'{output}'+'.p', 'wb')
my_path = Path('..', '..', f'{folder_out}', 'results_processed', f'{output}'+'.p') 
with my_path.open('wb') as fp:
    pickle.dump(CCS_ElcCap_diff, fp)

/var/folders/g0/db564x8n6j14t8scrcvbrmsc0000gn/T/ipykernel_33928/1685421715.py:2: FutureWarning: The default dtype for empty Series will be 'object' instead of 'float64' in a future version. Specify a dtype explicitly to silence this warning.
  CCS_ElcCap_diff = pd.Series(index=range(2020,2051))


In [33]:
#%% read in GLUCOSE data: AccumulatedAnnualDemand - Food - Crops, Livestock
# food consumption
Demand_MFOO = pd.DataFrame(columns=YEAR.VALUE).fillna(0)
Demand_VFOO = pd.DataFrame(columns=YEAR.VALUE).fillna(0)

variable = 'AccumulatedAnnualDemand'
csv_file = Path('..', '..', f'{folder_in}', 'data_csv', f'{variable}.csv')
AccumulatedAnnualDemand = pd.read_csv(csv_file)
MFOO = AccumulatedAnnualDemand.loc[(AccumulatedAnnualDemand["FUEL"]=='MFOO')]
VFOO = AccumulatedAnnualDemand.loc[(AccumulatedAnnualDemand["FUEL"]=='VFOO')]
for year in YEAR.VALUE:
    MFOO_temp = MFOO[(MFOO['YEAR'] == year)]
    VFOO_temp = VFOO[(VFOO['YEAR'] == year)]
    Demand_MFOO.loc[0, year] = MFOO_temp['VALUE'].sum()
    Demand_VFOO.loc[0, year] = VFOO_temp['VALUE'].sum()

In [34]:
# save GLUCOSE data: AccumulatedAnnualDemand - Food - Crops, Livestock into file
output1 = 'GLUCOSE_FoodDemand_VFOO'
output2 = 'GLUCOSE_FoodDemand_MFOO'

#fp = open(f'{output1}'+'.p', 'wb')
my_path = Path('..', '..', f'{folder_out}', 'results_processed', f'{output}'+'.p') 
with my_path.open('wb') as fp:
    pickle.dump(Demand_VFOO, fp)

#fp = open(f'{output2}'+'.p', 'wb')
my_path = Path('..', '..', f'{folder_out}', 'results_processed', f'{output}'+'.p') 
with my_path.open('wb') as fp:
    pickle.dump(Demand_MFOO, fp)

In [35]:
#%% read in GLUCOSE results: TotalAnnualTechnologyActivityByMode - Land Use, Forest land
# total forest land (LF1+LF2)
Forest_tot = pd.Series(index=YEAR.VALUE).fillna(0)

LF_tot = LandResources['LF1']+LandResources['LF2']
Forest_tot = LF_tot.T
print(Forest_tot)

VALUE
2010    4000.000000
2011    4000.000000
2012    4000.000000
2013    3991.705321
2014    3968.086129
2015    3916.016260
2016    3851.504081
2017    3786.813419
2018    3722.003769
2019    3657.015636
2020    3591.908514
2021    3552.882852
2022    3514.156647
2023    3475.625885
2024    3437.425273
2025    3399.452333
2026    3362.581485
2027    3325.973841
2028    3289.767498
2029    3253.863534
2030    3218.402004
2031    3195.699881
2032    3174.223643
2033    3152.521478
2034    3132.035702
2035    3112.114597
2036    3092.786398
2037    3074.080748
2038    3056.766877
2039    3039.341773
2040    3023.375448
2041    3016.307250
2042    3010.033600
2043    3004.534733
2044    2999.911857
2045    2996.208771
2046    2993.471466
2047    2991.688734
2048    2991.768369
2049    2992.168421
2050    2993.742385
dtype: float64


/var/folders/g0/db564x8n6j14t8scrcvbrmsc0000gn/T/ipykernel_33928/603326993.py:3: FutureWarning: The default dtype for empty Series will be 'object' instead of 'float64' in a future version. Specify a dtype explicitly to silence this warning.
  Forest_tot = pd.Series(index=YEAR.VALUE).fillna(0)


In [36]:
# save GLUCOSE results: TotalAnnualTechnologyActivityByMode - Land Use, Forest land into file
output = 'GLUCOSE_LandUse_LF'

#fp = open(f'{output}'+'.p', 'wb')
my_path = Path('..', '..', f'{folder_out}', 'results_processed', f'{output}'+'.p') 
with my_path.open('wb') as fp:
    pickle.dump(Forest_tot, fp)

In [37]:
#%% read in GLUCOSE results: AnnualEmissions
# Total Emissions - CO2EQ and WATER
Emissions = {}

for e in emissions:
    Emissions[e] = pd.Series(index=YEAR.VALUE)
    csv_file = Path('..', '..', f'{folder_out}', 'results_csv', 'AnnualEmissions.csv')
    if not os.path.exists(csv_file):
        pass
    else:
        AnnualEmissions = pd.read_csv(csv_file)
        for year in YEAR.VALUE:
            em_temp = AnnualEmissions[(AnnualEmissions['EMISSION'] == e)&(AnnualEmissions['YEAR'] == year)]
            Emissions[e].loc[year] = em_temp['VALUE'].sum()

e_co2eq = Emissions['CO2EQ']
e_water = Emissions['WATER']

/var/folders/g0/db564x8n6j14t8scrcvbrmsc0000gn/T/ipykernel_33928/1915406326.py:6: FutureWarning: The default dtype for empty Series will be 'object' instead of 'float64' in a future version. Specify a dtype explicitly to silence this warning.
  Emissions[e] = pd.Series(index=YEAR.VALUE)
/var/folders/g0/db564x8n6j14t8scrcvbrmsc0000gn/T/ipykernel_33928/1915406326.py:6: FutureWarning: The default dtype for empty Series will be 'object' instead of 'float64' in a future version. Specify a dtype explicitly to silence this warning.
  Emissions[e] = pd.Series(index=YEAR.VALUE)


In [38]:
# save GLUCOSE results: AnnualEmissions into file
output = 'GLUCOSE_Emissions'

#fp = open(f'{output}'+'.p', 'wb')
my_path = Path('..', '..', f'{folder_out}', 'results_processed', f'{output}'+'.p') 
with my_path.open('wb') as fp:
    pickle.dump(Emissions, fp)

In [39]:
#%% read in GLUCOSE results: AnnualTechnologyEmission - CO2EQ, Direct Air Capture technologies
# DAC penetration (C1ENDA1CS + C1ENDA2CS)
DAC_emission = pd.Series(index=YEAR.VALUE).fillna(0)

csv_file = Path('..', '..', f'{folder_out}', 'results_csv', 'AnnualTechnologyEmission.csv')
if not os.path.exists(csv_file):
    pass
else:
    AnnualTechnologyEmission = pd.read_csv(csv_file)
    for year in YEAR.VALUE:
        DAC_CO2EQ = AnnualTechnologyEmission[((AnnualTechnologyEmission['TECHNOLOGY'] == 'C1ENDA1CS')|(AnnualTechnologyEmission['TECHNOLOGY'] == 'C1ENDA2CS'))&(AnnualTechnologyEmission['EMISSION'] == 'CO2EQ')&(AnnualTechnologyEmission['YEAR'] == year)]
        DAC_emission.loc[year] = DAC_CO2EQ['VALUE'].sum()


/var/folders/g0/db564x8n6j14t8scrcvbrmsc0000gn/T/ipykernel_33928/976633855.py:3: FutureWarning: The default dtype for empty Series will be 'object' instead of 'float64' in a future version. Specify a dtype explicitly to silence this warning.
  DAC_emission = pd.Series(index=YEAR.VALUE).fillna(0)


In [40]:
output = 'GLUCOSE_Emissions_DAC'

#fp = open(f'{output}'+'.p', 'wb')
my_path = Path('..', '..', f'{folder_out}', 'results_processed', f'{output}'+'.p') 
with my_path.open('wb') as fp:
    pickle.dump(DAC_emission, fp)

In [41]:
# calculate technology penetration rate: AnnualTechnologyEmission - CO2EQ, Direct Air Capture technologies
DAC_emission_diff = pd.Series(index=range(2020,2051))
for c in DAC_emission_diff.index:
    DAC_emission_diff[c]=DAC_emission[c]-DAC_emission[c-1]

# save GLUCOSE results: AnnualTechnologyEmission - CO2EQ, Direct Air Capture technologies penetration rate into file
output = 'GLUCOSE_Emissions_DAC_diff'

#fp = open('..', '..', f'{folder_out}', 'results_processed', f'{output}'+'.p', 'wb')
my_path = Path('..', '..', f'{folder_out}', 'results_processed', f'{output}'+'.p') 
with my_path.open('wb') as fp:
    pickle.dump(DAC_emission_diff, fp)

/var/folders/g0/db564x8n6j14t8scrcvbrmsc0000gn/T/ipykernel_33928/4112921003.py:2: FutureWarning: The default dtype for empty Series will be 'object' instead of 'float64' in a future version. Specify a dtype explicitly to silence this warning.
  DAC_emission_diff = pd.Series(index=range(2020,2051))
